# Week 2 tutorial: Logistic regression par deux

### External files included for this exercise
 - $\texttt{ex2data1.txt}$ - Dataset for linear regression with one variable
 - $\texttt{ex2data2.txt}$ - Dataset for linear regression with multiple variables
 - Week_2_tute_P1 - Part one of this week's exercise.
 - Week_2_tute_P2 - Part two of this week's exercise.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

## 2 - Regularized logistic regression
In this part of the exercise, you will implement regularized logistic regression to predict whether microchips from a fabrication plant passes quality assurance (QA). During QA, each microchip goes through various tests to ensure it is functioning correctly.
Suppose you are the product manager of the factory and you have the test results for some microchips on two different tests. From these two tests, you would like to determine whether the microchips should be accepted or rejected. To help you make the decision, you have a dataset of test results on past microchips, from which you can build a logistic regression model.

### 2.1 - Visualizing the data
**The following import and visualization does not include a first column of ones as an intercept term.**<br>
Examining the data using an adaptation of the plotting function from above, this shows that our dataset cannot be separated into positive and negative examples by a straight-line through the plot. Therefore, a straight-forward application of logistic regression will not perform well on this dataset since logistic regression will only be able to find a linear decision boundary.

In [ ]:
data = np.loadtxt('./data/ex2data2.txt',delimiter=',')
X1 = data[:,0]
X2 = data[:,1]
m = X1.shape[0]
y = data[:,-1]
pos = [idx for (idx,val) in enumerate(y) if val==1]
neg = [idx for (idx,val) in enumerate(y) if val==0]
fig = plt.figure(figsize=(6,6))
plt.plot(X1[pos],X2[pos],'kx',MarkerSize=8,label='Accepted')
plt.plot(X1[neg],X2[neg],'ko',MarkerFaceColor='yellow',MarkerSize=8,label='Rejected')
plt.xlabel("Microchip test 1")
plt.ylabel("Microchip test 2")
plt.legend()
plt.show()

### 2.2 - Feature mapping
One way to fit the data better is to create more features from each datapoint. In the below function `map_feature()`, we can map the features into all polynomial terms of $x_1$ and $x_2$ up to the $n^\mathrm{th}$ power (we'll use $n=6$).
\begin{equation}
\mathrm{map\_feature}(x) = \begin{bmatrix}
x_1\\
x_2\\
x_1^2\\
x_1x_2\\
x_2^2 \\
x_1^3\\
\vdots\\
x_1x_2^5\\
x_2^6\\
\end{bmatrix}
\end{equation}
As a result of this mapping, our vector of two features (the scores on two QA tests) has been transformed into a 27-dimensional vector.  A logistic regression classifier trained on this higher-dimension feature vector will have a more complex decision boundary and will appear nonlinear when drawn in our 2-dimensional plot. *Note that we've excluded the 0$^{th}$ order term, as this intercept will come in separately.*<br>



While the feature mapping allows us to build a more expressive classifier, it also more susceptible to overfitting. In the next parts of the exercise, you will implement regularized logistic regression to fit the data and also see for yourself how regularization can help combat the overfitting problem.

In [ ]:
def map_feature(X1,X2,degree=6):
    '''
    Returns a matrix of feature vectors of the nth degree polynomial terms
    of two variables. Does not include constant intercept term. 
    
    Inputs
    ===============
    X1 : (m,1) vector of feature 1
    X2 : (m,1) vector of feature 2
    degree : degree of polynomial features
    
    Returns
    ===============
    out: (m,n) new matrix of training data
    '''
    new_size = (degree+2)*(degree+1) //2-1
    out = np.zeros((X1.shape[0],new_size))
    idx=0
    for i in range(1,degree+1):
        for j in range(0,i+1):
            out[:,idx] = X1**(i-j) * X2**j
            idx+=1
    return out
X = map_feature(X1,X2)

### 2.3 - Cost function and gradient
**Regularization** introduces a penalty term for specific weights becoming too large. This effectively reduces the learning algorithm's ability to over-fit a function, and favors a smooth decision boundary. Below, I will reiterate the functional form of the loss and gradient; however, we will opt for convenience and use scikit-learn for implementation. 

The regularized cost funciton in logistic regression is the cross-entropy loss with an added term:
\begin{equation}
J(\theta) = \frac{1}{m}\sum^m_{i=1} \left[
-y^{(i)} \log(h_\theta(x^{(i)})) - (1-y^{(i)}) \log(1-h_\theta(x^{(i)}))
\right]
+ \frac{\lambda}{2m} \sum^n_{j=1}\theta_j^2
\end{equation}
We do not regularize the intercept term, which is why it was excluded from our implementation of `map_feature()`. The gradient of the cost function is a vector where the $j^{\mathrm{th}}$ element is defined as:
\begin{equation}
\frac{\partial J(\theta)}{\partial\theta_j} = \left(\frac{1}{m} \sum^m_{i=1} (h_\theta(x^{(i)}) - y^{(i)})x^{(i)}_j\right) + \frac{\lambda}{2m}\theta_j
\end{equation}

#### 2.3.1 - Implementation with sklearn
You should feel confident that you could program this gradient descent on logistic regression with regularization. For the sake of breivity, we will use a handy machine learning library to do this for us. <br>
**Excercise :**
- Following along from the Exam Scores example, create a call to the class `LogistiRegression()`.
- In the initializer, specify the arguments `fit_intercept=True`, `C=1`. C is the inverse of $\lambda$: smaller values specify stronger regularization.
    - A sensible line would look something like `log_reg = LogisticRegression(fit_intercept=True, C=1)
- Next call the `.fit()` method on our variables `X` and `y`. 
- Print the training accuracy using the `.score()` method.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

### 2.4 - Understanding our results. 
Below, I've given you another cell that creates a lovely graph to examine the results of our algorithm. 
<br>
*Remember, all the machine learns is a function of probability with resepct to our high dimensional input. Anything with a probability less than 0.5 is a 'negative' example, and above such is a positive example.* <br>
**Exercise: Lets have a play with regularization.** The implementation of regularization in `sklearn.linear_model.LogisticRegression` uses a value of `C` that is the inverse of `lambda` in our equations above. Run the above and below cells with the following values for `C`:
- C = 1 | $\lambda$ = 1 : Appropriate regularization
- C = 0.01 | $\lambda$ = 100 : Too much regularization, *i.e.* poor fit. 
- C = 1e5 | $\lambda$ = 1e-5 : No regularization,  *i.e.* overfitting. 

Explore different values of regularization. In practice, we would train only on a portion of our data, and use a second portion to test the predicability of our model. Models that are overfit may have a high accuracy; however, they have limited predictive capacity and are thus useless. 

In [ ]:
xx, yy = np.mgrid[-1:1.2:0.01, -1:1.2:0.01]
grid = map_feature(xx.ravel(), yy.ravel())

probs = log_reg.predict_proba(grid)[:, 1].reshape(xx.shape)

f, ax = plt.subplots(figsize=(8, 6))
contour = ax.contourf(xx, yy, probs, 25, cmap="RdBu",
                      vmin=0, vmax=1)
ax_c = f.colorbar(contour)
ax_c.set_label("$P(y = 1)$")
ax_c.set_ticks([0, .25, .5, .75, 1])

ax.contour(xx, yy, probs, levels=[0.5],vmin=0, vmax=1,colors='green')

ax.scatter(X[:,0], X[:, 1], c=y[:], s=50,
           cmap="RdBu", vmin=-.2, vmax=1.2,
           edgecolor="white", linewidth=1)

ax.set(aspect="equal",
       xlim=(-1, 1.2), ylim=(-1, 1.2),
       xlabel="$X_1$", ylabel="$X_2$")
plt.show()